<font size="1.5">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.

<font size="1.5">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.


# UAVSAR SLC stack processor

<font size="3">The notebook uses functions from ISCE StripmapStack and some additional functionalities in uavsar_utils folder to create Interferograms from SLC data. 

<font size="3">Currently there is no support to automatically fetch the data. The users are required to copy the download urls from the [UAVSAR webpage](https://uavsar.jpl.nasa.gov/) into a text file locally. 
- If multiple segments exists, download urls for all the segments are needed.
- The notebook reuires ISCE environment. (The last three cells require ISCE + MintPy). 

In [ ]:
import os
import numpy as np
from osgeo import gdal
import glob
import shelve
import isce
import isceobj
import multiprocessing
import subprocess
import sys
import time
from pathlib import Path
n_processes = 4 #int(multiprocessing.cpu_count()) - 2;


## Add the required paths
<font size="3">Add path to the ISCE stripmapStack directory and the uavsar_utils directory

In [ ]:
#get the path of the ISCE base dir
env_str = os.popen("conda env list | grep '/isce$' | awk '{print $NF}'")
isce_base_dir = env_str.read().strip();
isce2_share_dir = f"{isce_base_dir}/share/isce2";

# now add stripmapStack contrib directory to PATH 
stripmapStack_dir = f"{isce2_share_dir}/stripmapStack";
# add UAVSAR functionalities to the path
uavsar_utils_dir = '/home/jovyan/atbd-se-development/uavsar_utils'
os.environ['PATH'] = f"{uavsar_utils_dir}:{stripmapStack_dir}:{os.environ['PATH']}"
os.environ['ATBD'] = '/home/jovyan/atbd-se-development/'
sys.path.append(os.environ['ATBD'])
!export OMP_NUM_THREADS=16


In [ ]:
#import insar functions. 
try:
    from uavsar_utils.insar import ImageReader, Interferogram, correlation
except:
    print('Add uavsar_utils directory to pythonpath')


In [ ]:
def run_cmd(cmd):
    print("Command: {}".format(cmd))
    try:
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print("Command failed with error: {}".format(e))

## Place all your inputs here. 
1. Point to your scratch directory or where you would process the data
2. Give a folder name for the processing. 
3. Specify number of looks and number of connections per SLC.
4. A bounding box is required to download the DEM. If not given will be calculated later from the metadata.

In [ ]:
scratch_dir = '/scratch/bvarugu'; #define your scratch dir
track_name = 'track_14003' #define the UAVSAR directory name
segments = [1,2,3,4]; #define the SLC segments; Used for concatenation
#reference_date = #input a reference date if not the first date will be considered reference date

rlks = 3 #define range looks
alks = 12  #define azimuth looks
numConnections = 2 #define number of interferogram pairs to be made with each SLC similar to ISCE
#bbox = '35 37 -122 -120' #Bounding box for DEM download #selected based on Northern San Andreas

out_dir = os.path.join(scratch_dir,track_name);
os.makedirs(out_dir, exist_ok=True);
download_dir= os.path.join(out_dir,'download');
os.makedirs(download_dir, exist_ok=True);
os.chdir(download_dir);


## Write the download links
- Create a file named download_links.sh in the download folder and copy all your links in it. UAVSAR webpage has 'wget' populated infront of the links already.


In [ ]:
#copy the links from UAVSAR webpage into a file in the download dir named download_links.sh
#this cell shall be updated if a better method of downloading UAVSAR data is available
print('Downloading UAVSAR SLCs using wget links')
!sh download_links.sh > download_log
!rm *.ann.*
!rm *.dop.*
os.chdir(out_dir);

## Create SLC directory for  one/multiple SLC segments
- A directory for each segment (Ex:SLC_seg1) is created. If only one segment then SLC directory will be created. 

In [ ]:
#Create SLC directory
dop_file = glob.glob(os.path.join(download_dir, '*.dop'))[0];

if len(segments)>1:
    print('Multiple Segments exist. Creating multiple SLC segments directories!')
    for i in range(len(segments)):
        slc_seg_dir = 'SLC_seg{}'.format(segments[i]);
        cmd = 'prepareUAVSAR_coregStack.py -i {} -d {} -o {} -s {}'.format(download_dir,dop_file,slc_seg_dir,segments[i]);
        print(cmd);
        os.system(cmd);
        if i< len(segments)-1:
            cmd = 'cp {}/*/*.ann {}'.format(slc_seg_dir,download_dir);
            print(cmd);
            os.system(cmd);
    slc_dir = os.path.join(out_dir,'SLC_seg{}'.format(segments[0])); 
else:
    print('Procesing only One Segment. No Concatenation')
    slc_dir = os.path.join(out_dir,'SLC');
    cmd = 'prepareUAVSAR_coregStack.py -i {} -d {} -o {} -s {}'.format(download_dir,dop_file,slc_dir,segments[0]);
    os.system(cmd);

print('SLC directory:',slc_dir);
dateList = sorted(os.listdir(slc_dir));
print('SLC dates:',dateList);


### Give a reference data if not the first data will be considered reference. 

In [ ]:
slc_dir = os.path.join(out_dir,'SLC_seg{}'.format(segments[0])); 
try:
    reference_date
except:
    reference_date=sorted(os.listdir(slc_dir))[0];
print('Reference date for Meta data:',reference_date);
reference_slc_dir = os.path.join(slc_dir,reference_date);

## Read Metadata from the reference SLC

- Here SLC meatadata is loaded into a dictionary variable to query for samples (columns), rows and bounding box for DEM.Each annotation file contains the info for all the segments.

In [ ]:
#read necessary info from the UAVSAR annotation file. Each annotation file contains the info for all the segments.
def read_uavsar_ann_file(ann_file_path):
    '''read UAVSAR ann file as a dict to query for rows, columns info '''
    result_dict = {}
    with open(ann_file_path, 'r') as file:
        for line in file:
            # Strip any leading/trailing whitespace and skip empty lines
            line = line.strip()
            if not line or line.startswith(';'):
                continue
            
            # Split the line into key and value using '=' as the separator
            if '=' in line:
                key, value = line.split('=', 1);
                key = key.split('(')[0].strip()
                value = value.split(';')[0].strip()
                result_dict[key.strip()] = value.strip()
    
    return result_dict
ann_file_path = glob.glob(os.path.join(reference_slc_dir, '*.ann'))[0];
slc_info_dict = read_uavsar_ann_file(ann_file_path);
key = 'slc_{}_1x1 Columns'.format(str(segments[0]))
samples = int(slc_info_dict[key]);
lines = 0;
for seg in segments:
    key = 'slc_{}_1x1 Rows'.format(str(seg))
    lines += int(slc_info_dict[key]);
print('total samples/columns:',samples);
print('total rows:',lines) 
lats= [];lons= [];
try:
    bbox
except:
    for seg in segments:
        for corner in [1,2,3,4]:
            lat, lon = slc_info_dict['Segment {} Data Approximate Corner 1'.format(seg,corner)].split(',')
            lats.append(float(lat));lons.append(float(lon));
    bbox = ' '.join(map(str,[int(np.min(lats)-0.1),int(np.max(lats)+0.1),int(np.min(lons)-0.1),int(np.max(lons)+0.1)]));
print('Bounding box for DEM:',bbox)

### Download SRTM DEM 

- If using own DEM is preferred, point to it as dem_file.

In [ ]:
#get DEM for geocoding
#if you have your DEM file deifne its path as dem_file '/path/to/dem_file' if not SRTM DEM will be used
#define bounding box for the DEM
try:
    dem_file
except:
    dem_dir = os.path.join(out_dir,'DEM');
    os.makedirs(dem_dir, exist_ok=True);
    os.chdir(dem_dir);
    source_link ="http://step.esa.int/auxdata/dem/SRTMGL1/"
    try:
        bbox
    except:
        print('Bounding box for DEM download required');
    cmd ='dem.py -a stitch -b {} -u {} -d {} -r -s 1 -c'.format(bbox,source_link,dem_dir);
    print(cmd);
    run_cmd(cmd);
    
dem_file = glob.glob(os.path.join(dem_dir, "*.dem.wgs84"))[0];
cmd = 'fixImageXml.py -f -i {}'.format(dem_file)
print(cmd);
os.system(cmd);
os.chdir(out_dir);

### Create interferogram pairs list
- Based on the numConnections variable input earlier. 

In [ ]:
#create pairs
looks = [alks, rlks];
ifg_dir =  os.path.join(out_dir,'Igrams');
os.makedirs(ifg_dir, exist_ok=True);
pairs_list= [];
for ii,dd in enumerate(dateList):
    referenceDate = dd;
    for jj in range(ii+1, ii+1+numConnections):
        if jj < len(dateList):
            secondaryDate = dateList[jj];
            ifg_pair =  referenceDate+'_'+secondaryDate;
            pairs_list.append(ifg_pair);


## Generate Interferometic products
- The code multilooks the SLCs and creates interferograms. Processes multiple segments simultenously. 

In [ ]:
#generate interferograms
def generate_isce_xml(ifg_file, amp_file, coh_file, samples):
    outInt = isceobj.Image.createIntImage()
    outInt.setFilename(ifg_file)
    outInt.setWidth(samples)
    outInt.setAccessMode('read')
    outInt.renderHdr()
    outInt.renderVRT()

    outAmp = isceobj.Image.createAmpImage()
    outAmp.setFilename(amp_file)
    outAmp.setWidth(samples)
    outAmp.setAccessMode('read')
    outAmp.renderHdr()
    outAmp.renderVRT()

    outCor = isceobj.Image.createImage()
    outCor.setFilename(coh_file)
    outCor.setWidth(samples)
    outCor.setAccessMode('read')
    outCor.setDataType('FLOAT')
    outCor.renderHdr()
    outCor.renderVRT()

def cat_and_interfere(slc_dir,samples,looks,segments,ifg_dir,pair):
    referenceDate,secondaryDate = pair.split('_');
    int_dir  = os.path.join(ifg_dir,referenceDate+'_'+secondaryDate);os.makedirs(int_dir, exist_ok=True);
    intf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.int');
    ampf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.amp');
    cohf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.coh');
    igram = Interferogram(samples, looks);
    int_samples = samples//looks[1];

    if len(segments)>1:
        with open(intf, 'wb') as fint, open(ampf, 'wb') as famp:
            for seg in segments:
                ref_img = slc_dir[:-1]+str(seg)+'/'+referenceDate+'/'+referenceDate+'.slc';#print(ref_img)
                sec_img = slc_dir[:-1]+str(seg)+'/'+secondaryDate+'/'+secondaryDate+'.slc';#print(sec_img)
                img0 = ImageReader(ref_img, samples, dtype='complex64')
                img1 = ImageReader(sec_img, samples, dtype='complex64')
                #log.info('... %s * conj(%s)', img0.filename, img1.filename)
                for int_row, amp_row in igram.iterrows(img0, img1):
                    int_row.tofile(fint);
                    amp_row.tofile(famp);
        correlation(intf, ampf, cohf);
        generate_isce_xml(intf, ampf, cohf, int_samples)
    else:
        ref_img = slc_dir+'/'+referenceDate+'/'+referenceDate+'.slc';#print(ref_img)
        sec_img = slc_dir+'/'+secondaryDate+'/'+secondaryDate+'.slc';#print(sec_img)
        with open(intf, 'wb') as fint, open(ampf, 'wb') as famp:
            img0 = ImageReader(ref_img, n, dtype='complex64')
            img1 = ImageReader(sec_img, n, dtype='complex64')
            #log.info('... %s * conj(%s)', img0.filename, img1.filename)
            for int_row, amp_row in igram.iterrows(img0, img1):
                    int_row.tofile(fint);
                    amp_row.tofile(famp);
        correlation(intf, ampf, cohf);
        generate_isce_xml(intf, ampf, cohf, int_samples)
with multiprocessing.Pool(processes=12) as pool:
    pool.starmap(cat_and_interfere,[(slc_dir,samples,looks,segments,ifg_dir,pair) for pair in pairs_list]);

## Unwrap the interferograms using SNAPHU

In [ ]:
#unwrap interferograms

def unwrap_snaphu(unwrap_method,defo_max,looks,slc_dir,ifg_dir,pair):
    referenceDate,secondaryDate = pair.split('_');
    reference_slc_dir = slc_dir+'/'+referenceDate
    int_dir  = os.path.join(ifg_dir,referenceDate+'_'+secondaryDate);
    intf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.int');
    unwf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.unw');
    cohf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.coh');
    cmd = ['unwrap.py -i {} -c {} -u {} -s {}/ -a {} -r {} -d {} -m {}'.format(intf,cohf,unwf,reference_slc_dir,looks[0],looks[1],defo_max,unwrap_method)];
    run_cmd(cmd)
unwrap_method = 'snaphu';
defo_max = 2;
for pair in pairs_list:
    unwrap_snaphu(unwrap_method, defo_max, looks, slc_dir, ifg_dir, pair);
    
#####Currently not working, crashing the instance ########
# with multiprocessing.Pool(processes=12) as pool:
#     pool.starmap(unwrap_snaphu,[(unwrap_method,defo_max,looks,slc_dir,ifg_dir,pair) for pair in pairs_list]);
# if __name__ == "__main__":
#     with ProcessPoolExecutor(max_workers=12) as executor:
#         executor.map(unwrap_snaphu, pairs_list)

### Generate baselines

In [ ]:
#create UAVSAR baselines
baselines_dir = os.path.join(out_dir,'baselines');
os.makedirs(baselines_dir, exist_ok=True);
cmd ='uavsar_baselines.py -s {} -b {}  -m {}'.format(slc_dir,baselines_dir,reference_slc_dir);
print(cmd);
run_cmd(cmd);


### Generate geometry files from the DEM

In [ ]:
#generate geometry files
geometery_dir = os.path.join(out_dir,'geometry'+str(alks)+'x'+str(rlks));
cmd = 'topo.py -a {} -r {} -d {} -m {} -o {} -n'.format(alks, rlks,dem_file,reference_slc_dir,geometery_dir);
print(cmd);
run_cmd(cmd);

### Copy referenceShelve directory from the master interferogram

In [ ]:
#create referenceShelve dir
reference_dir = os.path.join(out_dir,'referenceShelve');
reference_ifg = sorted(glob.glob(ifg_dir+'/'+reference_date+'_*'))[0];
print('Reference interferogram is:',reference_ifg)
cmd ='cp -r {}/referenceShelve {}'.format(reference_ifg,reference_dir)
print(cmd);
run_cmd(cmd);

## Prepare interferograms and geometry files to be loaded into MintPy.
- This step requires an environment with both ISCE and MintPy.

In [ ]:
#Prepare .rsc files for unwrapped interferograms to be loaded into MintPy
cmd = 'prep_isce.py -f {}/*/{} -m {}/data -b {}  -g {}'.format(ifg_dir,'*.unw',reference_dir,baselines_dir,geometery_dir)
print(cmd);
run_cmd(cmd);

### Add paths pointing the inteferometric and geometry files to the smallbaselineApp.cfg file

In [ ]:

mintpy_dir= os.path.join(out_dir,'mintpy');
os.makedirs(mintpy_dir, exist_ok=True);
config_file = Path(mintpy_dir)/('smallbaselineApp.cfg')
config_file_parameters = """
####mintpy parameters
mintpy.load.processor = isce
mintpy.compute.numWorker = auto
##path to interferograms
mintpy.load.autoPath = no
mintpy.load.metaFile       = {out_dir}/referenceShelve/data
mintpy.load.baselineDir    = {out_dir}/baselines  
mintpy.load.unwFile        = {out_dir}/Igrams/*/*.unw_snaphu.unw  
mintpy.load.corFile        = {out_dir}/Igrams/*/*.coh
mintpy.load.connCompFile   = {out_dir}/Igrams/*/*.unw_snaphu.conncomp  
#mintpy.load.intFile        = {out_dir}/Igrams/*/*.int  
###path to geometry files
mintpy.load.demFile = {out_dir}/{geometery_dir}/hgt.rdr
mintpy.load.lookupYFile = {out_dir}/{geometery_dir}/lat.rdr
mintpy.load.lookupXFile = {out_dir}/{geometery_dir}/lon.rdr
mintpy.load.incAngleFile = {out_dir}/{geometery_dir}/los.rdr
mintpy.load.azAngleFile = {out_dir}/{geometery_dir}/los.rdr
mintpy.load.shadowMaskFile = {out_dir}/{geometery_dir}/shadowMask.msk
""".format(out_dir=out_dir,geometery_dir=geometery_dir)
config_file.write_text(config_file_parameters);
print('MintPy config file:\n    {}:'.format(config_file))
print(config_file.read_text())

## Load files to make MintPy input data cube

In [ ]:
os.chdir(mintpy_dir);
cmd = 'smallbaselineApp.py ' + 'smallbaselineApp.cfg' + ' --dostep load_data'
print(cmd);
run_cmd(cmd);